In [1]:
import re
import numpy as np
import pandas as pd
import openpyxl

import geopandas as gpd
#from shapely.geometry import Point
#from shapely import wkt
#import geographiclib
#from geographiclib.geodesic import Geodesic
#import utm
#from shapely.ops import nearest_points

import sys
import os
import requests
from bs4 import BeautifulSoup
from lxml import html
import warnings
import unidecode

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

#import urbanpy as up
#import contextily as ctx

import io
import json

import plotly.graph_objects as go
import plotly.express as px
import plotly as py

pd.set_option('display.max_rows',999)
pd.set_option('display.min_rows',4)
pd.set_option('display.float_format', '{:,.2f}'.format)

warnings.filterwarnings("ignore")

In [2]:
path = r'c:/Users/Franco/dropbox/files/'

In [3]:
cw = pd.read_excel(path+'clean/cwpops.xlsx',index_col=[0,1]).loc[:,['pops','City2']]

In [4]:
idx = pd.IndexSlice

## Mex

In [149]:
pmex = 'Homicidios/mex/'

* https://www.gob.mx/sesnsp/acciones-y-programas/datos-abiertos-de-incidencia-delictiva
* https://drive.google.com/file/d/15IwT5yJ-fEF6tLc0iM4-Q1OWhOGRFnLB/view?usp=sharing

In [5]:
files_mex = os.listdir(path+mex)

In [10]:
mex = pd.read_excel(path+mex+files_mex[1])

In [51]:
mex['Homicidios_año'] = mex.loc[:,['Enero','Febrero','Marzo','Abril','Mayo','Junio','Julio','Agosto','Septiembre','Octubre','Noviembre','Diciembre']].sum(axis=1)

In [73]:
mex['City'] = mex['Municipio']

In [86]:
mex = mex.set_index('Entidad')
mex.at['Ciudad de México','City'] = 'Valle de Mexico'

In [97]:
mex1 = mex[(mex['Subtipo de delito'] == 'Homicidio doloso')].groupby(['Entidad','City']).agg({'Año':'first','Cve. Municipio':'first','Clave_Ent':'first','Municipio':'first','Homicidios_año':'sum'}).reset_index().rename(columns={'Entidad':'Estado'})

In [98]:
mex1['City'] = mex1['City'].apply(lambda x: unidecode.unidecode(x))

In [105]:
mex2 = cw.loc['Mexico','pops'].to_frame().join(mex1.set_index('City'),how='inner')

In [120]:
dup = mex2[mex2.loc[:,'Estado'].groupby('City').count()>1]

In [128]:
dup = dup.reset_index().loc[[1,5,8,9,12]]

In [135]:
mex3 = mex2[~mex2.index.duplicated(keep=False)].append(dup.set_index('City'))

In [144]:
mex3['Homicidios cada 100.000 hab.'] = mex3['Homicidios_año'] / (mex3['pops'] * 1000000 / 100000)

In [152]:
mex3.to_excel(path+pmex+'mex_2016.xlsx')

## Arg

In [403]:
homicidios_arg = pd.read_excel('c:\\Users\\Franco\\Dropbox\\files\\Homicidios\\arg\\homicidios_ar.xlsx',index_col=[0,1,2])

In [404]:
caba = pd.DataFrame({'Provincia':['Ciudad Autonoma de Buenos Aires'],
              'Departamento':['Ciudad de Buenos Aires'],
              'Year':[2016],
              'provincia_id':[np.nan],'dpto_id':[np.nan],
              'codigo_delito_agrupado_nombre':['Total de víctimas de homicidios dolosos'],
              'cantidad_hechos':[np.nan],
               'cantidad_victimas':[148]}).set_index(['Provincia','Departamento','Year'])

In [405]:
hom_arg = homicidios_arg.append(caba)

In [406]:
hom_arg = hom_arg.loc[idx[:,:,2016],:]

In [407]:
hom_arg = hom_arg.reset_index()

In [408]:
hom_arg['City'] = hom_arg['Departamento']

In [409]:
hom_arg['City'] = hom_arg['City'].apply(lambda x: unidecode.unidecode(x))
hom_arg['Provincia'] = hom_arg['Provincia'].apply(lambda x: unidecode.unidecode(x))
hom_arg['Departamento'] = hom_arg['Departamento'].apply(lambda x: unidecode.unidecode(x))

In [410]:
pgba = ['Avellaneda',
'General San Martín',
'Hurlingham',
'Ituzaingó',
'José C. Paz',
'Lanús',
'Lomas de Zamora',
'Malvinas Argentinas',
'Morón',
'Quilmes',
'San Isidro',
'Tres de Febrero',
'Vicente López',
'Almirante Brown',
'Berazategui',
'Esteban Echeverría',
'Ezeiza',
'Florencio Varela',
'La Matanza',
'Merlo',
'Moreno',
'San Fernando',
'San Miguel',
'Tigre']

In [411]:
pgba = [unidecode.unidecode(x) for x in pgba]

In [412]:
hom_arg = hom_arg.set_index(['Provincia','Departamento'])

In [414]:
hom_arg.loc[idx['Buenos Aires',pgba],'City'] = 'Partidos del GBA'

In [415]:
hom_arg = hom_arg.reset_index().set_index(['Provincia','City'])

In [416]:
amba = hom_arg.loc[('Buenos Aires','Partidos del GBA'),:]

In [417]:
noamba = hom_arg.drop(index=amba.index)

In [418]:
amba = amba.reset_index().groupby('City').agg({'Provincia':'first','Year':'first','provincia_id':'first','dpto_id':'first','cantidad_hechos':'sum','cantidad_victimas':'sum'}).reset_index().set_index(['Provincia','City'])

In [419]:
ha = noamba.append(amba)

In [498]:
ha['Cityh'] = ha.index

In [420]:
city_cw = cw.loc['Argentina'].index.to_list()

In [458]:
cityh = [('Buenos Aires','Bahia Blanca'),
 ('Ciudad Autonoma de Buenos Aires','Ciudad de Buenos Aires'),
 ('Chubut','Escalante'),
 ('Entre Rios','Concordia'),
 ('Corrientes','Capital'),
 ('Formosa','Formosa'),
 ('Catamarca','Capital'),
 ('Cordoba','Capital'),
 ('Buenos Aires','La Plata'),
 ('Mendoza','Capital'),
 ('Entre Rios','Parana'),
 ('Chaco','San Fernando'),
 ('Santa Fe','Rosario'),
 ('San Juan','Capital'),
 ('Santa Fe','La Capital'),
 ('Tucuman','Capital'),
 ('Jujuy','Dr. Manuel Belgrano'),
 ('La Rioja','Capital'),
 ('Buenos Aires','General Pueyrredon'),
 ('Neuquen','Confluencia'),
 ('Buenos Aires','Partidos del GBA'),
 ('Misiones','Capital'),
 ('Chubut','Rawson'),
 ('Cordoba','Rio Cuarto'),
 ('Santa Cruz','Guer Aike'),
 ('Salta','Capital'),
 ('San Luis','La Capital'),
 ('Buenos Aires','San Nicolas'),
 ('Mendoza','San Rafael'),
 ('La Pampa','Centro (Santa Rosa)'),
 ('Santiago del Estero','Capital'),
 ('Buenos Aires','Tandil'),
 ('Tierra del Fuego','Ushuaia'),
 ('Rio Negro','Adolfo Alsina')]

In [486]:
merge = pd.DataFrame({'City':city_cw,'Cityh':cityh}).set_index('City')

In [478]:
cwa = cw.loc['Argentina'].reset_index()

In [479]:
cwa['City2'] = cwa['City']

In [489]:
cwa = cwa.set_index('City').join(merge)

In [504]:
cwa = cwa.reset_index().merge(ha,left_on='Cityh',right_on='Cityh',how='left',indicator=True).set_index('City')

In [507]:
cwa['Homicidios cada 100.000 hab.'] = cwa['cantidad_victimas'] / (cwa['pops'] * 1000000 / 100000)

In [508]:
cwa.to_excel('c:\\Users\\Franco\\Dropbox\\files\\Homicidios\\arg\\hom_arg_final.xlsx')

## Bra

In [83]:
hbra_tasa = pd.read_excel('c:\\Users\\Franco\\Dropbox\\files\\Homicidios\\bra\\homicidios_bra.xlsx',index_col=[0,1])

In [87]:
hom = hbra_tasa.reset_index().rename(columns={'City':'nome'})

In [88]:
hom['City'] = hom['nome'].apply(lambda x: unidecode.unidecode(x))

In [89]:
hom = hom.sort_values(['City','Year']).set_index(['City','Year'])

In [90]:
h16 = hom.loc[idx[:,2016],:]

In [91]:
cbra = cw.loc['Brasil']

In [95]:
hbra = cbra.join(h16,how='inner').sort_values(['City','Homicidios cada 100.000 hab.'],ascending=[True,False]).drop_duplicates('City2',keep='first')#.groupby('City').agg({'pops':'first','City2':'first':'cod'})

In [96]:
hbra.to_excel('c:\\Users\\Franco\\Dropbox\\files\\Homicidios\\bra\\homicidos_2016.xlsx')

col

In [97]:
homicidios_col = pd.read_excel('c:\\Users\\Franco\\Dropbox\\files\\Homicidios\\col\\homicidios_col.xlsx',index_col=[0,1])

In [101]:
col = cw.loc['Colombia']

In [105]:
homicidios_col.join(col,how='inner')

,,Departamento,Homicidios cada 100.000 hab.,pops,City2
City,Year,,,,
Armenia,2016,Quindío,36.22,2.96,Armenia
Barrancabermeja,2016,Santander,16.69,0.16,Barrancabermeja
Barranquilla,2016,Atlántico,31.95,1.27,Barranquilla
Bogota,2016,Cundinamarca,16.02,7.41,Bogota
Bucaramanga,2016,Santander,18.74,0.58,Bucaramanga
Cali,2016,Valle del Cauca,53.78,2.23,Cali
Cartagena,2016,Bolívar,23.78,0.91,Cartagena
Cartago,2016,Valle del Cauca,33.84,0.11,Cartago
Cucuta,2016,Norte de Santander,40.22,0.75,Cucuta


In [398]:
hcol = homicidios_col.reset_index().loc[homicidios_col.reset_index()['City']=='Cali',:].drop(columns=['Year','Departamento']).set_index('City')

## Uni

In [134]:
homicidios_col = homicidios_col.drop(index='Otros')

In [136]:
homicidios_col['Country'] = 'Colombia'

In [138]:
homicidios_col = homicidios_col.reset_index().set_index(['Country','City'])

In [146]:
homicidios_col.head()

Year Departamento  Homicidios cada 100.000 hab.
Country  City                                                            
Colombia Aguachica        2016        Cesar                         22.36
         Apartado         2016    Antioquia                         20.14
         Arauca           2016       Arauca                         13.38
         Armenia          2016      Quindío                         36.22
         Barrancabermeja  2016    Santander                         16.69

In [112]:
cwa = pd.read_excel('c:\\Users\\Franco\\Dropbox\\files\\Homicidios\\arg\\hom_arg_final.xlsx').set_index(['City','Year'])

In [140]:
cwa['Country'] = 'Argentina'

In [141]:
cwa = cwa.reset_index().set_index(['Country','City'])

In [142]:
cwa.head()

Year  pops  \
Country   City                                        
Argentina Bahia Blanca-Cerri             2016  0.31   
          Ciudad de Buenos Aires         2016  3.08   
          Comodoro Rivadavia-Rada Tilly  2016  0.24   
          Concordia                      2016  0.17   
          Corrientes                     2016  0.35   

                                                                 City2  \
Country   City                                                           
Argentina Bahia Blanca-Cerri                        Bahia Blanca-Cerri   
          Ciudad de Buenos Aires                Ciudad de Buenos Aires   
          Comodoro Rivadavia-Rada Tilly  Comodoro Rivadavia-Rada Tilly   
          Concordia                                          Concordia   
          Corrientes                                        Corrientes   

                                                                                     Cityh  \
Country   City                                                                               
Argentina Bahia Blanca-Cerri                              ('Buenos Aires', 'Bahia Blanca')   
          Ciudad de Buenos Aires         ('Ciudad Autonoma de Buenos Aires', 'Ciudad de...   
          Comodoro Rivadavia-Rada Tilly                            ('Chubut', 'Escalante')   
          Concordia                                            ('Entre Rios', 'Concordia')   
          Corrientes                                             ('Corrientes', 'Capital')   

                                                   Departamento  provincia_id  \
Country   City                                                                  
Argentina Bahia Blanca-Cerri                       Bahia Blanca          6.00   
          Ciudad de Buenos Aires         Ciudad de Buenos Aires           nan   
          Comodoro Rivadavia-Rada Tilly               Escalante         26.00   
          Concordia                                   Concordia         30.00   
          Corrientes                                    Capital         18.00   

                                          dpto_id  \
Country   City                                      
Argentina Bahia Blanca-Cerri             6,056.00   
          Ciudad de Buenos Aires              nan   
          Comodoro Rivadavia-Rada Tilly 26,021.00   
          Concordia                     30,015.00   
          Corrientes                    18,021.00   

                                                   codigo_delito_agrupado_nombre  \
Country   City                                                                     
Argentina Bahia Blanca-Cerri             Total de víctimas de homicidios dolosos   
          Ciudad de Buenos Aires         Total de víctimas de homicidios dolosos   
          Comodoro Rivadavia-Rada Tilly  Total de víctimas de homicidios dolosos   
          Concordia                      Total de víctimas de homicidios dolosos   
          Corrientes                     Total de víctimas de homicidios dolosos   

                                         cantidad_hechos  cantidad_victimas  \
Country   City                                                                
Argentina Bahia Blanca-Cerri                         nan                 10   
          Ciudad de Buenos Aires                     nan                148   
          Comodoro Rivadavia-Rada Tilly              nan                 23   
          Concordia                                  nan                 17   
          Corrientes                                 nan                  5   

                                        _merge  Homicidios cada 100.000 hab.  
Country   City                                                                
Argentina Bahia Blanca-Cerri              both                          3.23  
          Ciudad de Buenos Aires          both                          4.81  
          Comodoro Rivadavia-Rada Tilly   both                          9.49  

In [116]:
hmex = pd.read_excel(r'C:/Users/Franco/Dropbox/files/Homicidios/mex/mex_2016.xlsx').rename(columns={'Año':'Year'}).set_index(['City','Year'])

In [143]:
hmex['Country'] = 'Mexico'

In [144]:
hmex = hmex.reset_index().set_index(['Country','City'])

In [145]:
hmex.head()

Year  pops                           Estado  \
Country City                                                          
Mexico  Acayucan        2016  0.10  Veracruz de Ignacio de la Llave   
        Acuna           2016  0.11             Coahuila de Zaragoza   
        Aguascalientes  2016  0.18                   Aguascalientes   
        Campeche        2016  0.19                         Campeche   
        Celaya          2016  0.28                       Guanajuato   

                        Cve. Municipio  Clave_Ent       Municipio  \
Country City                                                        
Mexico  Acayucan                 30003         30        Acayucan   
        Acuna                     5002          5           Acuña   
        Aguascalientes            1001          1  Aguascalientes   
        Campeche                  4002          4        Campeche   
        Celaya                   11007         11          Celaya   

                        Homicidios_año  Homicidios cada 100.000 hab.  
Country City                                                          
Mexico  Acayucan                    35                         33.98  
        Acuna                       12                         11.09  
        Aguascalientes              27                         14.89  
        Campeche                    16                          8.39  
        Celaya                      98                         35.28

In [148]:
hbra['Country'] = 'Brasil'

In [150]:
hbra = hbra.reset_index().set_index(['Country','City'])

In [152]:
homicidios = homicidios_col.append(hmex).append(cwa).append(hbra).dropna(axis=1)

In [154]:
homicidios.to_excel(r'c:/users/franco/dropbox/files/clean/homicidios.xlsx')